# Extended HATA model

In [ ]:
def path_loss_variation(distance):
    
    mu = 12 # https://github.com/edwardoughton/pysim5g/blob/5a8e50b98a292de10bf9a0c0977cbd8411e8596d/src/pysim5g/path_loss.py#L336
    
    if distance <= 0.04:
        
        sigma = 3.5
        
    elif 0.04 < distance <= 0.1:
        
        sigma = 3.5 + ((12 - 3.5)/(0.1 - 0.04)) * (distance - 0.04)
        
    elif 0.1 < distance <= 0.2:
        
        sigma = 12
        
    elif 0.2 < distance <= 0.6:
        
        sigma = 12 + ((9 - 12)/(0.6 - 0.2)) * (distance - 0.2)
        
    else:
        
        sigma = 12

    normal_std = np.sqrt(math.log10(1 + (sigma/mu)**2))
    normal_mean = math.log10(mu) - normal_std**2 / 2

    path_loss_variation = np.random.lognormal(normal_mean, normal_std, 1)[0]

    return(path_loss_variation)

In [ ]:
def alpha_value(frequency, distance, bts_height):
    
    if distance < 20:
    
        alpha = 1
    
    else:
    
        alpha = 1 + (0.14 + 1.87*10**(-4)*frequency + 1.07*10**(-3)*bts_height)*(math.log10(distance/20))**0.8
        
    return(alpha)

In [ ]:
def path_loss_short(frequency, distance, bts_height, ms_height):
    
    path_loss = 32.4 + 20*math.log10(frequency) + 10*math.log10(distance**2 + (bts_height - ms_height)**2/10**6)
    
    return(path_loss)

In [ ]:
def path_loss_long(frequency, settlement_type, distance, bts_height, ms_height, alpha):
    
    if frequency == 900:
        
        path_loss_long = 69.6 \
                            + 26.2 * math.log10(frequency) \
                            - 13.82 * math.log10(max(30, bts_height)) \
                            + (44.9 - 6.55 * math.log10(max(30, bts_height))) * (math.log10(distance))**alpha \
                            - ((1.1 * math.log10(frequency) - 0.7) * min(10,ms_height) - (1.56 * math.log10(frequency) - 0.8) + max(0, 20 * math.log10(ms_height / 10))) \
                            - (min(0, 20 * math.log10(bts_height / 30)))
            
    else:
            
        path_loss_long = 46.3 \
                            + 33.9 * math.log10(2000) + 10*math.log10(frequency / 2000)\
                            - 13.82 * math.log10(max(30, bts_height)) \
                            + (44.9 - 6.55 * math.log10(max(30, bts_height))) * (math.log10(distance))**alpha \
                            - ((1.1 * math.log10(frequency) - 0.7) * min(10, ms_height) - (1.56 * math.log10(frequency) - 0.8) + max(0, 20 * math.log10(ms_height / 10))) \
                            - (min(0, 20 * math.log10(bts_height / 30)))
        
    if (settlement_type == 'suburban') | (settlement_type == 2):
        
        path_loss = path_loss_long \
                            - 2 * (math.log10((min(max(150, frequency), 2000)) / 28))**2 \
                            - 5.4
    
    elif (settlement_type == 'rural') | (settlement_type == 1):
        
        path_loss = path_loss_long \
                            - 4.78 * (math.log10(min(max(150, frequency), 2000)))**2 \
                            + 18.33 * math.log10(min(max(150, frequency), 2000)) \
                            - 40.94
        
    else:
        
        path_loss = path_loss_long
        
    return(path_loss)

In [ ]:
def path_loss_mid(frequency, settlement_type, distance, bts_height, ms_height, alpha):
    
    path_loss = path_loss_short(frequency = frequency, distance = 0.04, bts_height = bts_height, ms_height = ms_height) \
                + ((math.log10(distance) - math.log10(0.04))/(math.log10(0.1) - math.log10(0.04))) \
                * (path_loss_long(frequency = frequency, settlement_type = settlement_type,distance = 0.1, bts_height = bts_height, ms_height = ms_height, alpha = alpha) \
                               - path_loss_short(frequency = frequency, distance = 0.04, bts_height = bts_height, ms_height = ms_height))
    
    return(path_loss)

In [ ]:
def calculate_path_loss_hata(frequency, settlement_type, distance, bts_height, ms_height, alpha):

    if distance < 0.04:

        path_loss = path_loss_short(frequency, distance, bts_height, ms_height)

    elif 0.04 <= distance < 0.1:
        
        path_loss = path_loss_mid(frequency, settlement_type, distance, bts_height, ms_height, alpha)
        
    else:
        
        path_loss = path_loss_long(frequency, settlement_type, distance, bts_height, ms_height, alpha)
    
    return(path_loss + path_loss_variation(distance))

In [ ]:
def calculate_path_loss_hata_novar(frequency, settlement_type, distance, bts_height, ms_height, alpha):

    if distance < 0.04:

        path_loss = path_loss_short(frequency, distance, bts_height, ms_height)

    elif 0.04 <= distance < 0.1:
        
        path_loss = path_loss_mid(frequency, settlement_type, distance, bts_height, ms_height, alpha)
        
    else:
        
        path_loss = path_loss_long(frequency, settlement_type, distance, bts_height, ms_height, alpha)
    
    return(path_loss + 2.94)